# Factorization Machines
論文リンク：https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf   
今回は[fastFM](https://github.com/ibayer/fastFM)というライブラリを使用して実装

In [ ]:
#!pip instal fastFM

In [ ]:
import os
from time import time
import numpy as np
import pandas as pd
from fastFM import sgd
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


## データの入力
fastFMで二値分類をする場合ラベルを{+1, -1}にする必要があるためデータセットのラベルを変換しなくてはいけない

In [ ]:
# 単に{+1,-1}にするための関数
def map_rating(x):
    if x == 0:
        return -1
    else:
        return 1
    

In [ ]:
# メモリに乗り切らない場合はサンプルを減らす
# 後のDictVectorizerで全量ないと予測・評価でコケるため一旦全てを結合する

train_df = pd.read_csv('../data/MovieLens20M/classification/train20m.csv', nrows=20000)
eval_df = pd.read_csv('../data/MovieLens20M/classification/eval20m.csv', nrows=20000)
test_df = pd.read_csv('../data/MovieLens20M/classification/test20m.csv', nrows=20000)

# fastFMのfit時に改めてtrain_test_splitされるためtrainとevalはこの後も一緒にする
dataset = pd.concat([train_df, eval_df])
dataset = pd.concat([dataset, test_df])
test_size = len(test_df)

print('Test data size: {}'.format(test_size))

dataset.rating = dataset.rating.map(lambda x: map_rating(x))
dataset.head()

del train_df
del eval_df
del test_df


In [ ]:
# DictVectorizer用にdictのlistを作成
X_list = []

# 目的変数はDictVectorizerにいれないのでそのまま分割
# yは一次元にreshapeする（.reshape(-1,1)とかすると実行中にコケる）
y_train = np.array(dataset.iloc[:-test_size,-1]).reshape(-1,)
y_test = np.array(dataset.iloc[-test_size:,-1]).reshape(-1,)

t1 = time()
for row in dataset.iloc[:,:3].itertuples(index=False, name=None):
    X_list.append({"user": str(int(row[0])), "item": str(int(row[1]))})
t2 = time()
print('Finished in {:.4f} seconds'.format(t2-t1))
del dataset


In [ ]:
print(len(X_list))

In [ ]:
v = DictVectorizer()
X = v.fit_transform(X_list)

X_train = X[:-test_size]
X_test = X[-test_size:]
del X_list


## 学習

In [ ]:
fm = sgd.FMClassification(n_iter=1000000, init_stdev=0.1, l2_reg_w=0,
                          l2_reg_V=0.01, rank=40, step_size=0.1)
fm.fit(X_train, y_train)


## 予測・評価

In [ ]:
y_pred = fm.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, np.round(y_pred))
print('AUC: {:.4f}\nAccuracy: {:.4f}'.format(auc, accuracy))
